# Tutorial: (Stochastic) Poisson Surface Reconstruction

This notebook will show how to use (Stochastic) Poisson Surface Reconstruction [[1](https://hhoppe.com/poissonrecon.pdf),[2](https://www.dgp.toronto.edu/projects/stochastic-psr/)] to reconstruct a surface from an oriented point cloud using Python and `gpytoolbox` [[3](https://gpytoolbox.org)].  

All prerequisites can be installed with pip and conda.
<!-- ```bash
pip install gpytoolbox=0.0.3
pip install numpy
pip install scipy
pip install matplotlib
``` -->

### Prerequisites

We can install all prerequisites with pip

## Poisson Surface Reconstruction [[1](https://hhoppe.com/poissonrecon.pdf)]

The input to this algorithm is an oriented point cloud: a set of tuples $\{(p_i,\vec{n}_i)\}_{i=1}^n$ where $p_i\in\mathbb{R}^d$ are points on an object's surface and $\vec{n}_i\in\mathbb{R}^d$ are the corresponding surface normals at each point. In `gpytoolbox`, we will store this type of data in two matrices $\mathbf{P},\mathbf{N}\in\mathbb{R}^{n\times d}$, such that the i-th row of $\mathbf{P}$ contains the coordinates $p_i$ and the i-th row of $\mathbf{N}$ contains the coordinates of $\vec{n}_i$. Let's begin by generating and plotting a simple example point cloud using basic `numpy` and `matplotlib`.

## Stochastic Poisson Surface Reconstruction [[2](https://www.dgp.toronto.edu/projects/stochastic-psr/)]
Poisson Surface Reconstruction outputs a scalar function and, through it, a possible reconstructed surface. However, it ignores a key fact: surface reconstruction is, fundamentally, an under-determined problem, and there are infinitely many possible reconstructed surfaces. *Stochastic* Poisson Surface Reconstruction formalizes this intuition by instead of computing a function $f:\mathbb{R}^d\rightarrow \mathbb{R}$, modeling it as a Gaussian distribution
$$f(x)\sim \mathcal{N}(\mu(x),\sigma(x))$$
and computing the functions $\mu:\mathbb{R}^d\rightarrow \mathbb{R}$ and $\sigma:\mathbb{R}^d\rightarrow \mathbb{R}^+$. In `gpytoolbox`, it's as easy as including the `stochastic=True` flag in our call to `poisson_surface_reconstruction`:


While we are obtaining the correct values for $v(x)$ and $s(x)$ at the grid points, it is a bit disappointing that they look so pixelated when plotting. This is specially clear in the last example, where the surface probability is at times so thinly supported that its support falls below the grid spacing and gives the appearance that there is no surface when plotted. Using `gpytoolbox`, we can actually plot the values of these statistical queries on a finer grid:

### Subspace reduction
Using a larger grid is a much clearer way of visualizing the computed statistical quantities that have features below the scale of grid resolution. Of course, another option would be to carry out the whole computation on a larger grid, but this will have a larger associated computational cost. When this is the case (this is especially necessary in 3D applications), we can use a space reduction technique (see Section 4.2. in [[2](https://www.dgp.toronto.edu/projects/stochastic-psr/stochastic-psr-lr.pdf)]) to reduce the computational cost. In the `gpytoolbox` implementation, this is done by adding a `solve_subspace_dim` argument. The higher the value, the lower the approximation error will be at a higher computational cost. In practice, values between 1,000 and 10,000 are reasonable.

### 3D examples
(Stochastic) Poisson Surface Reconstruction is dimension agnostic, and `gpytoolbox`'s implementation also allows us to use it to compute surfaces in three dimensions. However, the functions $f, \mu, \sigma$ are harder to visualize in this case. One slightly hacky way of doing it is to use the tetrahedral mesh visualizer in `polyscope` [[4](https://polyscope.run/py/)].


In [23]:
# Generate a point cloud
from gpytoolbox import read_mesh, per_face_normals, random_points_on_mesh, poisson_surface_reconstruction
import numpy as np

v,f = read_mesh("example-scenes/gp-tree/tree.obj")
print(v.shape,f.shape)
n = 5000
P,I,_ = random_points_on_mesh(v,f,n,return_indices=True)
N = per_face_normals(v,f)[I,:]

print(P.shape,N.shape)

minp = P.min(axis=0)
maxp = P.max(axis=0)
center = (minp + maxp) * 0.5
extends = (maxp - minp) * 0.5

gs = 1*np.array([64,64,64]) #44
# Call stochastic PSR
corner = center - extends*1.5
h = extends*3/gs[0]

print(gs, corner, h)
scalar_mean, scalar_var, grid_vertices = poisson_surface_reconstruction(P,N,gs=gs,corner=corner,h=h,solve_subspace_dim=1000,stochastic=True)



(1303930, 3) (1599403, 3)
(5000, 3) (5000, 3)
[64 64 64] [-3.48755583 -1.22031066 -3.70824749] [0.10965757 0.11390315 0.11056473]


In [22]:
# There's no integration with Jupyter notebooks, but here's what you could run to visualize the result in polyscope:

####
from gpytoolbox import regular_cube_mesh
import polyscope as ps
tet_verts, tets = regular_cube_mesh(gs[0],type='hex')
tet_verts = tet_verts[:,(2,0,1)]
tet_verts = tet_verts * h * gs[0] + corner
print(tet_verts.shape)
ps.init()
ps_vol = ps.register_volume_mesh("test volume mesh", tet_verts, hexes=tets, enabled=False)
ps_vol.add_scalar_quantity("mean", scalar_mean)
ps_vol.add_scalar_quantity("sigma", scalar_var)
sample_points = ps.register_point_cloud("sample points", P)
sample_points.add_vector_quantity("sample normals", N, enabled=True)
ps.show()
####

(32768, 3)


In [2]:
# The relevance of the variance is easier to see if we make the surface incomplete
N = N[P[:,0] < 0,:]
P = P[P[:,0] < 0,:]
scalar_mean, scalar_var, grid_vertices = poisson_surface_reconstruction(P,N,gs=gs,corner=corner,h=h,solve_subspace_dim=1000,stochastic=True)



In [3]:
print(scalar_mean.shape)

scalar_mean.tofile("testing/spsr/bunny-64-mean.bin")
scalar_var.tofile("testing/spsr/bunny-64-var.bin")

grid_vertices = np.array(grid_vertices)
print(grid_vertices.transpose().reshape(-1,3).shape)
grid_vertices.transpose().reshape(-1,3).tofile("testing/spsr/bunny-64-grid_vertices.bin")

(262144,)
(262144, 3)


In [11]:
# There's no integration with Jupyter notebooks, but here's what you could run to visualize the result in polyscope:

####
from gpytoolbox import regular_cube_mesh
from skimage.measure import marching_cubes

import polyscope as ps
tet_verts, tets = regular_cube_mesh(gs[0],type='hex')
tet_verts = 2.2*tet_verts - 1.1
R = np.array([[0.0,0.0,1.0],[0.0,1.0,0.0],[-1.0,0.0,0.0]]) @ np.array([[1.0,0.0,0.0],[0.0,0.0,1.0],[0.0,-1.0,0.0]])
tet_verts = tet_verts @ R
tet_verts[:,0] = - tet_verts[:,0]
tet_verts[:,1] = - tet_verts[:,1]
ps.init()
ps_vol = ps.register_volume_mesh("test volume mesh", tet_verts, hexes=tets, enabled=False)
ps_vol.add_scalar_quantity("mean", scalar_mean)
ps_vol.add_scalar_quantity("sigma", scalar_var)
sample_points = ps.register_point_cloud("sample points", P)
sample_points.add_vector_quantity("sample normals", N, enabled=True)

verts, faces, normals, values = marching_cubes(np.reshape(scalar_mean,(gs[0],gs[1],gs[2]),order='F'),level=0.0)
# skimage's marching cubes outputs a mesh in the [0,gs] interval, so we need to rescale it
verts = verts/(gs-1)
verts = 2.2*verts - 1.1

ps_vol = ps.register_surface_mesh("mean_mesb", verts, faces=faces, enabled=False)
ps_mesh = ps.register_surface_mesh("original_mesh", v, faces=f, enabled=True)

ps.show()
####

# References
[1] "[Poisson Surface Reconstruction](https://hhoppe.com/poissonrecon.pdf)" by *Michael Kazhdan, Matthew Bolitho and Hugues Hoppe*, 2006.  

[2] "[Stochastic Poisson Surface Reconstruction](https://www.dgp.toronto.edu/projects/stochastic-psr/)" by *Silvia Sellán and Alec Jacobson*, 2022.  

[3] "[Gpytoolbox: A Python Geometry Processing library](https://gpytoolbox.org)" by *Silvia Sellán and Oded Stein*, 2022.